In [1]:
import Pkg; 

if split(pwd(),"/")[end] == "utility"
    cd(joinpath(@__DIR__, "../../"))
    Pkg.activate("Project.toml")
end

using MorphoMol
using MorphoMolNotebooks
using StaticArrays
using Rotations
using Distances
using JLD2

  Activating project at `~/Doktor/Code/MorphoMol/MorphoMolNotebooks`


In [5]:
mol_type = "6r7m"
algo = "rwm"
energy = "op"
#folder = "../../Data/hpc_out/scans/2_6r7m/1/"
folder = "../../Data/hpc_out/$(algo)_$(energy)_2_$(mol_type)/"

id_index = 1
separator = "_"
max = 1
for file in readdir(folder)
    if split(file, ".")[end] == "jld2"
        max = max < parse(Int, split(file, separator)[id_index]) ? parse(Int, split(file, separator)[id_index]) : max
    end
end

acceptance_rates = Vector{Float64}([])
min_Es = [Inf for _ in 1:max]
evaluation_strings = ["" for _ in 1:max]
for file in readdir(folder)
    if split(file, ".")[end] == "jld2"
        try
            @load "$folder$file" input output
            # Crude test to exclude runs that remained dispersed
            if maximum(output["Es"]) - minimum(output["Es"]) < 10.0# || input["T"] > 1.5
                println("E_diff = $(maximum(output["Es"]) - minimum(output["Es"]))")
                continue
            end

            id = parse(Int, split(file, separator)[id_index])
            exp_template_centers = MorphoMol.Utilities.TWOTMVSU_EXPERIMENTAL_ASSEMBLY[mol_type]["template_centers"]
            exp_state = MorphoMol.Utilities.TWOTMVSU_EXPERIMENTAL_ASSEMBLY[mol_type]["state"]
            theta = MorphoMol.Utilities.average_offset_distance(exp_template_centers, input["template_centers"], exp_state, output["states"][argmin(output["Es"])])
            #string = "$(id): $(input["T"]) | $(input["overlap_slope"]) | $(output["αs"][end]) | $(output["Es"][1]), $(minimum(output["Es"])), $(output["Es"][end]) | $(theta)"
            string = "$(id): $(input["T"]) | $(output["αs"][end]) | $(output["Es"][1]), $(minimum(output["Es"])), $(output["Es"][end]) | $(theta)"

            push!(acceptance_rates, output["αs"][end])
            evaluation_strings[id] = string
            min_Es[id] = minimum(output["Es"])
        catch e
            println(e)
            println("Error in $file")
        end
    end
end
println("Minimal configuration id: $(argmin(min_Es)) | Average acceptance rate: $(sum(acceptance_rates) / length(acceptance_rates))")
for elem in evaluation_strings
    println(elem)
end

Minimal configuration id: 8 | Average acceptance rate: 0.4453233247929877

2: 1.075 | 0.4182713 | 11.668576779989827, -29.739446273867536, -28.834225746869073 | 4.540806177914487
3: 1.075 | 0.43799827 | 142.15393018431328, -29.48904420085499, -20.867674390627926 | 3.6140336665596973
4: 1.075 | 0.45917013 | -15.606171647321304, -28.99450336138328, -18.506808556184225 | 5.21963810634819
5: 1.075 | 0.47177535 | 49.96486441603283, -29.862871861340167, -25.34629520785301 | 9.006789912868792
6: 1.075 | 0.46954906 | 105.64127585277824, -29.531658432222386, -17.72096907750648 | 18.657719180228664
7: 1.075 | 0.45114762 | 40.75111750093241, -29.488199520055034, -19.3124121186235 | 0.5982458591349102
8: 1.075 | 0.43347052 | 18.116883703146854, -30.877681134385917, -20.52655761366516 | 9.00488876013423

10: 1.075 | 0.42831576 | 75.5279536279447, -29.63327070727889, -21.113825469526375 | 17.88554181755896
11: 1.075 | 0.4300094 | 54.58386841271383, -29.76733255974696, -22.722760329460748 | 3.9450808

In [ ]:
for file in readdir(folder)
    try
        @load "$folder$file" input output
        mindex = argmin(output["Es"])
        state = output["states"][mindex]
        id = parse(Int, split(file, "_")[id_index])
        radii = collect(Base.Iterators.flatten([input["template_radii"] for _ in 1:input["n_mol"]]))
        MorphoMolNotebooks.configuration_to_poly(MorphoMol.Utilities.get_flat_realization(state, input["template_centers"]), radii, "$(folder)$(id)", input["n_mol"], length(input["template_radii"]))
    catch e
        println(e)
        continue
    end
end

# Print Single Assembly Path

In [ ]:
@load "$(folder)/11_$(algo_e)_2_$(mol_type).jld2" input output

2-element Vector{Symbol}:
 :input
 :output

In [ ]:
"$(folder)/11_$(algo_e)_2_$(mol_type).jld2"

"../../Data/hpc_out/2_6r7m/rwm_ma_2_6r7m//11_rwm_ma_2_6r7m.jld2"

In [ ]:
using GeometryBasics

In [ ]:
mindex = argmin(output["Es"])
lower = 500
upper = 100
out_folder = "../../Data/tmp/assembly_path/"
try
    rm(out_folder, recursive=true)
catch 
end
mkdir(out_folder)
radii = collect(Base.Iterators.flatten([input["template_radii"] for _ in 1:input["n_mol"]]))
for (i,j) in enumerate(maximum([mindex-lower,1]):minimum([mindex+upper, length(output["Es"])]))
    state = output["states"][j]
    MorphoMolNotebooks.configuration_to_poly(MorphoMol.Utilities.get_flat_realization(state, input["template_centers"]), radii, "$(out_folder)$(i)", input["n_mol"], length(input["template_radii"]))
end



# Dispersed State Energies

In [ ]:
folder = "assets/input/poly/singles/"
for file in readdir(folder)
    c,r = MorphoMol.Utilities.poly_to_state("$folder$file")
    E = MorphoMol.Energies.solvation_free_energy(
        c, 
        length(r),
        r,
        1.4,
        MorphoMol.Energies.get_prefactors(1.4, 0.3665),
        0.0,
        0.85
        )
    println("$(split(file, "_")[1]) ", 2*E)
end

1ei7 8255.3356279267
2tmv 8130.426867538925
6r7m 8219.578304239454
6sae 8185.217724310747
6sag 8174.157936120334
